In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

In [ ]:
import os

os.environ['USE_PYGEOS'] = '0'  # remove geopandas warnings

import holoviews as hv
import numpy as np
import numpy_indexed as npi
import pandas as pd
import xarray as xr

hv.extension("bokeh")

In [ ]:
%%bash
pushd /tmp
wget --quiet https://github.com/ec-jrc/Thalassa/files/10867068/fort.63.zip 
unzip -o fort.63.zip
popd

In [ ]:
from thalassa import api

filename = "/tmp/fort.63.nc"
ds = api.open_dataset(filename)
ds

## Main API

`thalassa` supports several types of graphs, including:
    
- A visualization of the variables at various timestamps and or layers.
- A visualization of the mesh
- Extraction of timeseries from specific points

The following cell shows the basic usage:

In [ ]:
#variable, layer, timestamp = "salt", 40, ds.time.values[4]
#variable, layer, timestamp = "depth", None, None
variable, layer, timestamp = "zeta", None, "max"

# The trimesh is the most basic object. This is what you need to create all the others graphs
# It is on this object that you specify the timestamp and/or the layer.
trimesh = api.create_trimesh(ds=ds, variable=variable, timestamp=timestamp, layer=layer)

# The wireframe is the representation of the mesh
wireframe = api.get_wireframe(trimesh=trimesh)

# The tiles is using the tiling service from Open Street maps
tiles =  api.get_tiles() 

# The raster object is the basic Map that visualizes the variable. 
# You can specify things like the colorbar limits and/or the extents
#raster = api.get_raster(trimesh, clim_min=0, clim_max=15)
raster = api.get_raster(trimesh)

# The pointer/tap timeseries extract the timeseries of a specific node from the xr.Dataset and visualize it.
pointer_dmap = api.get_pointer_timeseries(ds=ds, variable=variable, source_raster=raster, layer=layer)
tap_dmap = api.get_tap_timeseries(ds=ds, variable=variable, source_raster=raster, layer=layer)

### Let's visualize!

After you render the layout, move the mouse over the map and click on it. This will fill in the `pointer_dmap` and the `tap_dmap`.

In [ ]:
layout = tiles * raster.opts(width=600, cmap="viridis") 
layout
(pointer_dmap + tap_dmap).opts(shared_axes=False)